In [1]:
import json
import os
import sys

sys.path.append('/home/luis/CGE/luxgiant-dstream/luxgiant_dstream')
#sys.path.append('/mnt/0A2AAC152AABFBB7/CGE/luxgiant-dstream')

from luxgiant_dstream.prep_ds import PrepDS
from luxgiant_dstream.gwas_fixed import GWASfixed
from luxgiant_dstream.gwas_random import GWASrandom

In [2]:
#params = json.load(open("/mnt/0A2AAC152AABFBB7/data/LuxGiantimputed/configFiles/parameters.JSON"))
#paths = json.load(open("/mnt/0A2AAC152AABFBB7/data/LuxGiantimputed/configFiles/paths_home_local.JSON"))

params = json.load(open("//home/luis/data/LuxGiantimputed/configFiles/parameters.JSON"))
paths = json.load(open("/home/luis/data/LuxGiantimputed/configFiles/paths_local.JSON"))

In [3]:
#prep = PrepDS(
#    input_path=paths['input_directory'],
#    input_name=paths['input_prefix'],
#    output_path=paths['output_directory'],
#    output_name=paths['output_prefix'],
#    config_dict=params,
#    dependables_path=paths['dependables_directory'],
#)

In [4]:
#prep_steps = {
#    'ld_prune': prep.exclude_high_ld_hla,
#    'pca'     : prep.pca_decomposition
#}
#
#for step in prep_steps.keys():
#    prep_steps[step]()

In [3]:
gwas_f = GWASfixed(
    input_path = paths['input_directory'],
    input_name = paths['input_prefix'],
    output_path= paths['output_directory'],
    output_name= paths['output_prefix'],
    dependables= paths['dependables_directory'],
    config_dict= params,
    preps_path = os.path.join(paths['output_directory'], 'preparatory'),
)
gwas_f.compare_gwas_fixed_file_name = 'european_cohort_nallsEtAl2019_excluding23andMe_allVariants.hg38.tsv'
gwas_f.compare_gwas_fixed_highlights= 'highlighted_european_cohort_nallsEtAl2019_excluding23andMe_allVariants.hg38.csv'

Analysis of GWAS data using a fixed model initialized.


In [4]:
gwas_f_steps = {
    #'train_model': gwas_f.fixed_model_association_analysis,
    #'top_hits'   : gwas_f.get_top_hits,
    #'annotate'   : gwas_f.annotate_top_hits,
    #'plots'      : gwas_f.plot_drawings,
    #'one_cohort' : gwas_f.one_cohort_comparison,
    'trumpet'     : gwas_f.create_trumpet_plot
}

for step in gwas_f_steps.keys():
    gwas_f_steps[step]()

FileNotFoundError: [Errno 2] No such file or directory: 'plink'

In [7]:
#gwas_r = GWASrandom(
#    input_path = paths['input_directory'],
#    input_name = paths['input_prefix'],
#    output_path= paths['output_directory'],
#    output_name= paths['output_prefix'],
#    config_dict= params,
#    preps_path = os.path.join(paths['output_directory'], 'preparatory'),
#)

In [8]:
#gwas_r_steps = {
#    'preparatory': gwas_r.prepare_aux_files,
#    'grm'        : gwas_r.compute_grm,
#    'random'     : gwas_r.run_gwas_random,
#    'top_hits'   : gwas_r.get_top_hits,
#    'annotate'   : gwas_r.annotate_top_hits,
#    'plots'      : gwas_r.plot_drawings
#}
#for step in gwas_r_steps.keys():
#    gwas_r_steps[step]()

In [1]:
import subprocess

# Define your PLINK command
plink_command = ["plink", "--help"]  # Replace with your actual PLINK command

# Run the command
result = subprocess.run(plink_command, capture_output=True, text=True)

FileNotFoundError: [Errno 2] No such file or directory: 'plink'